# Nationality Insights per Kiosk

This notebook explores nationality dynamics for each kiosk, including current distribution, historical trends, churn, and retention derived from `Retail contract IDs - Sheet1.csv` (updated through November 2025).



In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
sns.set_theme(style="whitegrid")

root = Path("/Users/IhabMatar/Desktop/Int Arimax")
data_path = root / "Retail contract IDs - Sheet1.csv"
clean_path = root / "analysis" / "contracts_kiosk_clean.csv"
monthly_path = root / "analysis" / "kiosk_nationality_monthly_active.csv"
summary_path = root / "analysis" / "kiosk_nationality_cleaning_summary.json"
metrics_path = root / "analysis" / "kiosk_nationality_time_series.csv"
latest_path = root / "analysis" / "kiosk_nationality_latest_distribution.csv"
recent_summary_path = root / "analysis" / "kiosk_nationality_recent_summary.csv"



In [ ]:
raw_df = pd.read_csv(
    data_path,
    parse_dates=["Contract Creation Date", "Paid Date", "Termination Date"],
)
clean_df = pd.read_csv(
    clean_path,
    parse_dates=["Contract Creation Date", "Paid Date", "Termination Date", "Start Date", "End Date"],
)
monthly_df = pd.read_csv(monthly_path, parse_dates=["Month"])
metrics_df = pd.read_csv(metrics_path, parse_dates=["Month"])
latest_df = pd.read_csv(latest_path, parse_dates=["Month"])
recent_df = pd.read_csv(recent_summary_path)
cleaning_summary = pd.read_json(summary_path, typ="series")
latest_month = metrics_df["Month"].max()
latest_df = latest_df.sort_values(["Kiosk", "active_contracts"], ascending=[True, False])

latest_df.head()



## Data Overview & Quality Notes

The cleaning script (`analysis/contracts_kiosk_clean.csv`) standardizes kiosk and nationality labels, removes exact duplicates, and limits each `Contract ID` to its latest record. The summary below highlights the resulting coverage and gaps.


In [ ]:
cleaning_summary


In [ ]:
missing_nat = clean_df[clean_df["Client Nationality Clean"] == "Unknown"]
missing_nat.groupby("Kiosk").size().sort_values(ascending=False).head(10)


## Latest Month Distribution (per kiosk)

The table below lists the top nationalities per kiosk for the most recent month in the dataset (`2025-11`), along with their share of active contracts within the kiosk. Adjust `top_n` to explore broader tails if needed.



In [ ]:
top_n = 5
latest_top = (
    latest_df.groupby("Kiosk")
    .head(top_n)
    .assign(share_of_kiosk=lambda df: (df["share_of_kiosk"] * 100).round(1))
)
latest_top[["Kiosk", "Client Nationality", "active_contracts", "share_of_kiosk"]]



## Trend Analysis

The following cells illustrate how nationality mix evolves over time. Adjust the filters to focus on specific kiosks or nationalities of interest.


In [ ]:
focus_kiosk = "Festival Plaza"
# Identify top nationalities by cumulative activity
kiosk_slice = metrics_df[metrics_df["Kiosk"] == focus_kiosk]
top_nationalities = (
    kiosk_slice.groupby("Client Nationality")["active_contracts"].sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)
trend = kiosk_slice[kiosk_slice["Client Nationality"].isin(top_nationalities)]
trend_pivot = trend.pivot_table(
    index="Month",
    columns="Client Nationality",
    values="active_contracts",
    aggfunc="sum",
    fill_value=0,
)
trend_pivot.tail()



In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
trend_pivot.plot(ax=ax)
ax.set_title(f"Top nationalities over time — {focus_kiosk}")
ax.set_ylabel("Active contracts")
ax.set_xlabel("Month")
plt.tight_layout()
plt.show()



## Churn & Retention

Churn is defined as contracts that terminated within a given month relative to the prior month's active base for the same kiosk-nationality. Retention reflects the share of customers that remained after churn. Values are capped at 0–100% to avoid distortion from small denominators.


In [ ]:
recent_months = metrics_df["Month"].sort_values().unique()[-3:]
recent_slice = metrics_df[metrics_df["Month"].isin(recent_months)]
churn_summary = (
    recent_slice.groupby(["Kiosk", "Client Nationality"])
    .agg(
        avg_churn_rate=("churn_rate", "mean"),
        avg_retention_rate=("retention_rate", "mean"),
        latest_active=("active_contracts", "last"),
        net_change_total=("net_change", "sum"),
    )
    .reset_index()
)
churn_summary.sort_values("avg_churn_rate", ascending=False).head(10)



In [ ]:
churn_summary.sort_values("net_change_total", ascending=False).head(10)


## Key Questions to Explore Further

- Which kiosks rely heavily on `Unknown` nationalities, and can onboarding capture these details more consistently?
- Are high-churn combinations concentrated around specific campaigns or seasonal locations?
- How do nationality-driven mixes intersect with revenue or contract value (if available)?

Document any answers or follow-ups in the notebook as analyses progress.
